In [1]:
import keras
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import TextVectorization,Embedding

In [2]:
train= pd.read_csv('train.csv')
test= pd.read_csv('test.csv')
train.shape

(7613, 5)

In [3]:
train.target.unique()

array([1, 0])

In [4]:
train

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...,...,...,...
7608,10869,NaN,NaN,Two giant cranes holding a bridge collapse int...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1
7610,10871,NaN,NaN,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,10872,NaN,NaN,Police investigating after an e-bike collided ...,1


In [5]:
test.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [6]:
train_shuffle= train.sample(frac=1,random_state=45)

In [7]:
train_shuffle.head()

,id,keyword,location,text,target
6761,9687,tornado,NaN,Heather Night and Ava Sparxxx enjoy a wild tee...,0
7196,10308,weapon,NaN,Pulse rifles after weapon tuning? http://t.co/...,0
1505,2173,catastrophic,"Quito, Ecuador.",Learning from the Legacy of a Catastrophic Eru...,1
6702,9600,thunder,"Macon, GA",#thunder outside my house this afternoon #gawx...,1
3028,4349,earthquake,Earth,1.43 earthquake occurred near Mount St. Helens...,1


In [8]:
from sklearn.model_selection import train_test_split

In [9]:
train_sent,val_sent,train_lab,val_lab= train_test_split(train_shuffle['text'].to_numpy(),
                                                        train_shuffle['target'].to_numpy(),test_size=0.1,random_state=45)

In [10]:
train_sent.shape,val_sent.shape,train_lab.shape,val_lab.shape

((6851,), (762,), (6851,), (762,))

In [11]:
#### TextVectorization
text_vec= TextVectorization(max_tokens=10000,
                            standardize='lower_and_strip_punctuation',
                            split='whitespace', output_mode='int',
                            output_sequence_length=15)

In [12]:
text_vec.adapt(train_sent)

In [13]:
sentence='There is flood in the city and cause more damage'

In [14]:
text_vec(sentence)

<tf.Tensor: shape=(15,), dtype=int64, numpy=
array([ 74,   9, 224,   5,   2, 195,   7, 340,  52, 235,   0,   0,   0,
         0,   0])>

In [15]:
#### Embedding
emb= Embedding(input_dim=10000,
              output_dim=128,input_length=15,
              embeddings_initializer="uniform")

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [16]:
emb(text_vec(sentence))

<tf.Tensor: shape=(15, 128), dtype=float32, numpy=
array([[ 0.03152383,  0.00749383,  0.00647137, ...,  0.01070918,
        -0.03642927,  0.03244026],
       [-0.00839276, -0.03746588,  0.04029154, ..., -0.04975302,
         0.01504174, -0.02045944],
       [-0.0313068 , -0.03982543, -0.04656764, ..., -0.02302442,
        -0.01523442,  0.04631789],
       ...,
       [ 0.0301428 , -0.01898012,  0.00104686, ...,  0.04337234,
         0.005689  , -0.00350205],
       [ 0.0301428 , -0.01898012,  0.00104686, ...,  0.04337234,
         0.005689  , -0.00350205],
       [ 0.0301428 , -0.01898012,  0.00104686, ...,  0.04337234,
         0.005689  , -0.00350205]], dtype=float32)>

RNN

In [17]:
rnn= tf.keras.models.Sequential()
rnn.add(tf.keras.Input(shape=(1,),dtype=tf.string))
rnn.add(text_vec)
rnn.add(emb)
rnn.add(tf.keras.layers.SimpleRNN(units=80,return_sequences=True))
rnn.add(tf.keras.layers.SimpleRNN(units=80))
rnn.add(tf.keras.layers.Dense(units=80,activation='relu'))
rnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
rnn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ (None, 15, 80)         │        16,720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_1 (SimpleRNN)        │ (None, 80)             │        12,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 80)             │         6,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,316,161 (5.02 MB)

 Trainable params: 1,316,161 (5.02 MB)

 Non-trainable params: 0 (0.00 B)

In [18]:
rnn.compile(optimizer=tf.keras.optimizers.Adam(),loss= tf.keras.losses.binary_crossentropy,metrics=['accuracy'])

In [19]:
history= rnn.fit(train_sent,train_lab,epochs=10,batch_size=30,validation_data=(val_sent,val_lab))

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 11s 31ms/step - accuracy: 0.6799 - loss: 0.5884 - val_accuracy: 0.8018 - val_loss: 0.4554
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8889 - loss: 0.2965 - val_accuracy: 0.7795 - val_loss: 0.5441
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step - accuracy: 0.9485 - loss: 0.1436 - val_accuracy: 0.7795 - val_loss: 0.6786
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9674 - loss: 0.0840 - val_accuracy: 0.7625 - val_loss: 0.8092
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 8s 33ms/step - accuracy: 0.9744 - loss: 0.0533 - val_accuracy: 0.7769 - val_loss: 0.8588
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.9764 - loss: 0.0517 - val_accuracy: 0.7402 - val_loss: 0.8728
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 7s 29ms/step - accuracy: 0.9784 - loss: 0.0412 - val_accuracy: 0.7336 - val_loss: 1.1232
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 25ms/step - accuracy: 0.9796 - loss: 0.0406 - val_ac

In [33]:
pred= rnn.predict(test['text'].to_numpy())
pred1=tf.squeeze(np.round(pred))

102/102 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step


In [43]:
pred= rnn.predict(val_sent)
pred1=tf.squeeze(np.round(pred))
pred1

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0.,
       1., 0., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
       0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0., 0

In [44]:
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

In [46]:
def calculate_results(true,pred):
  accuracy= accuracy_score(true,pred)
  precision,recall,f1,_= precision_recall_fscore_support(true,pred,average='weighted')
  model_results= {'accuracy':accuracy,
                  'precision':precision,
                  'recall':recall,
                  'f1_score':f1}
  return model_results

In [55]:
precision_recall_fscore_support(val_lab,pred1)

(array([0.75884956, 0.70645161]),
 array([0.79032258, 0.66768293]),
 array([0.77426637, 0.68652038]),
 array([434, 328]))

In [47]:
calculate_results(val_lab,pred1)

{'accuracy': 0.7375328083989501,
 'precision': 0.736295061675669,
 'recall': 0.7375328083989501,
 'f1_score': 0.736496438444073}

LSTM

In [49]:
lstm= tf.keras.models.Sequential()
lstm.add(tf.keras.Input(shape=(1,),dtype=tf.string))
lstm.add(text_vec)
lstm.add(emb)
lstm.add(tf.keras.layers.LSTM(units=80,return_sequences=True))
lstm.add(tf.keras.layers.LSTM(units=80))
lstm.add(tf.keras.layers.Dense(units=80,activation='relu'))
lstm.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
lstm.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ (None, 15, 80)         │        66,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ (None, 80)             │        51,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 80)             │         6,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,404,961 (5.36 MB)

 Trainable params: 1,404,961 (5.36 MB)

 Non-trainable params: 0 (0.00 B)

In [50]:
lstm.compile(optimizer=tf.keras.optimizers.Adam(),loss= tf.keras.losses.binary_crossentropy,metrics=['accuracy'])

In [52]:
history= lstm.fit(train_sent,train_lab,epochs=10,batch_size=30,validation_data=(val_sent,val_lab))

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 14s 44ms/step - accuracy: 0.8674 - loss: 0.3618 - val_accuracy: 0.7992 - val_loss: 0.6013
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 39ms/step - accuracy: 0.9489 - loss: 0.1590 - val_accuracy: 0.7572 - val_loss: 0.6169
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.9598 - loss: 0.1213 - val_accuracy: 0.7612 - val_loss: 0.8849
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 10s 45ms/step - accuracy: 0.9692 - loss: 0.0877 - val_accuracy: 0.7664 - val_loss: 1.1357
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.9761 - loss: 0.0580 - val_accuracy: 0.7480 - val_loss: 1.3413
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 11s 46ms/step - accuracy: 0.9810 - loss: 0.0445 - val_accuracy: 0.7651 - val_loss: 1.2141
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9790 - loss: 0.0467 - val_accuracy: 0.7454 - val_loss: 1.1307
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 8s 34ms/step - accuracy: 0.9779 - loss: 0.0475 - val_

In [57]:
pred= lstm.predict(val_sent)
pred2=tf.squeeze(np.round(pred))
pred2

24/24 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


<tf.Tensor: shape=(762,), dtype=float32, numpy=
array([1., 0., 1., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
       0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0.,
       0., 0., 0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0.,
       1., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
       1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1.,
       0., 0., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0

In [58]:
calculate_results(val_lab,pred2)

{'accuracy': 0.7506561679790026,
 'precision': 0.7525729056873343,
 'recall': 0.7506561679790026,
 'f1_score': 0.7512832902929388}

GRU

In [59]:
gru= tf.keras.models.Sequential()
gru.add(tf.keras.Input(shape=(1,),dtype=tf.string))
gru.add(text_vec)
gru.add(emb)
gru.add(tf.keras.layers.GRU(units=80,return_sequences=True))
gru.add(tf.keras.layers.GRU(units=80))
gru.add(tf.keras.layers.Dense(units=80,activation='relu'))
gru.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))
gru.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 15)             │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 15, 128)        │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 15, 80)         │        50,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 80)             │        38,880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 80)             │         6,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,375,841 (5.25 MB)

 Trainable params: 1,375,841 (5.25 MB)

 Non-trainable params: 0 (0.00 B)

In [60]:
gru.compile(optimizer=tf.keras.optimizers.Adam(),loss= tf.keras.losses.binary_crossentropy,metrics=['accuracy'])

In [61]:
history= gru.fit(train_sent,train_lab,epochs=10,batch_size=30,validation_data=(val_sent,val_lab))

Epoch 1/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 15s 42ms/step - accuracy: 0.8681 - loss: 0.2797 - val_accuracy: 0.7756 - val_loss: 0.8080
Epoch 2/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9719 - loss: 0.0656 - val_accuracy: 0.7402 - val_loss: 1.3421
Epoch 3/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9741 - loss: 0.0641 - val_accuracy: 0.7572 - val_loss: 1.3947
Epoch 4/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9771 - loss: 0.0444 - val_accuracy: 0.7559 - val_loss: 1.3928
Epoch 5/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 41ms/step - accuracy: 0.9796 - loss: 0.0453 - val_accuracy: 0.7493 - val_loss: 1.4620
Epoch 6/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 10s 42ms/step - accuracy: 0.9830 - loss: 0.0372 - val_accuracy: 0.7559 - val_loss: 1.5251
Epoch 7/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 11s 43ms/step - accuracy: 0.9832 - loss: 0.0318 - val_accuracy: 0.7572 - val_loss: 1.7543
Epoch 8/10
229/229 ━━━━━━━━━━━━━━━━━━━━ 9s 36ms/step - accuracy: 0.9815 - loss: 0.0377 - val_a

In [63]:
pred= lstm.predict(val_sent)
pred3=tf.squeeze(np.round(pred))

24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step


In [64]:
calculate_results(val_lab,pred3)

{'accuracy': 0.7545931758530183,
 'precision': 0.7543345710548723,
 'recall': 0.7545931758530183,
 'f1_score': 0.7544520836379902}